# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
weather_csv_file = "../output_data/city_weather_data.csv"
weather_df = pd.read_csv(weather_csv_file)
weather_df.head()


,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,caravelas,BR,-17.7125,-39.2481,77.99,79,86,4.56,1666716802
1,hermanus,ZA,-34.4187,19.2345,62.80,80,48,9.46,1666716803
2,santa maria,BR,-29.6842,-53.8069,78.10,50,0,11.50,1666716763
3,bluff,NZ,-46.6000,168.3333,43.56,86,100,10.11,1666716803
4,rikitea,PF,-23.1203,-134.9692,72.75,77,65,19.91,1666716804


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

locations = weather_df[["Lat", "Lon"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                           dissipating = False, max_intensity=max(weather_df["Humidity"]),
                           point_radius=2)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
#Save new dataframe to fit my conditions
perfect_weather = weather_df

#Filter based on conditions: temperature, cloudiness, wind
perfect_weather= perfect_weather.loc[(perfect_weather["Max Temp"] < 80) & (perfect_weather["Max Temp"] > 75)]
perfect_weather= perfect_weather.loc[perfect_weather["Wind Speed"] < 8]
perfect_weather= perfect_weather.loc[perfect_weather["Cloudiness"] == 0]

perfect_weather = perfect_weather.dropna()
perfect_weather.head()



,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date
46,neuquen,AR,-38.9516,-68.0591,78.69,19,0,6.91,1666716594
269,constantine,DZ,36.3650,6.6147,78.21,18,0,5.75,1666716921
313,ratnagiri,IN,16.9833,73.3000,75.88,73,0,5.91,1666716942
333,salalah,OM,17.0151,54.0924,78.08,64,0,3.80,1666716901
405,dalbandin,PK,28.8947,64.4101,75.16,15,0,2.53,1666716978


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#store into variable
hotel_df = perfect_weather
#print(hotel_df)
#set parameters
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    params["location"] = f"{lat},{lon}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
#print(json.dumps(response, indent=4, sort_keys=True))
#    print(results)    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (Keyerror, IndexError):
        print("Missing field/result...skipping.")
    print("------------")
          

Retrieving Results for Index 46: neuquen.
Closest hotel is Hotel del Comahue.
------------
Retrieving Results for Index 269: constantine.
Closest hotel is Grand Hotel Cirta.
------------
Retrieving Results for Index 313: ratnagiri.
Closest hotel is Hotel Landmark.
------------
Retrieving Results for Index 333: salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
------------
Retrieving Results for Index 405: dalbandin.
Closest hotel is Sanjrani House.
------------


In [14]:
hotel_df

,City,Country,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
46,neuquen,AR,-38.9516,-68.0591,78.69,19,0,6.91,1666716594,Hotel del Comahue
269,constantine,DZ,36.3650,6.6147,78.21,18,0,5.75,1666716921,Grand Hotel Cirta
313,ratnagiri,IN,16.9833,73.3000,75.88,73,0,5.91,1666716942,Hotel Landmark
333,salalah,OM,17.0151,54.0924,78.08,64,0,3.80,1666716901,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
405,dalbandin,PK,28.8947,64.4101,75.16,15,0,2.53,1666716978,Sanjrani House


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))